In [21]:
import os
import pandas as pd

"""
What does this do?
Split UCSC repeatmasker annotation into various subsets (classes or families).
Should be used for updating LOLA enrichment calculation / more fine-grained repeat enrichment
"""

path = '/home/local/work/code/github/project-diploid-assembly/annotation/grch38/grch38_rmsk.bed'

df = pd.read_csv(path, sep='\t', header=0)

discard_uncertain = (df['repClass'].str.contains('?', regex=False)) | (df['repFamily'].str.contains('?', regex=False))

df = df.loc[~discard_uncertain, :].copy()

select_main_chroms = 'chr[0-9XY]+$'

df = df.loc[df['#genoName'].str.match(select_main_chroms), :].copy()
df.reset_index(drop=True, inplace=True)
df.sort_values(['#genoName', 'genoStart'], inplace=True)
# should be 5293264
print(df.shape)


dump_families = [
    'SVA',
    'Low_complexity',
    'Simple_repeat',
    'centr',
    'acro',
    'telo',
    'Satellite',
    'Alu'
]

dump_classes = [
    'DNA',
    'RNA',
    'LTR',
    'LINE',
    'SINE'
]

outpath = '/home/local/work/data/hgsvc/lola/rmsk_new'

used_indices = set()
for fam in dump_families:
    select_family = df['repFamily'] == fam
    select_indices = ~df.index.isin(used_indices)
    sub = df.loc[select_family & select_indices, :]
    
    new_indices = set(sub.index)
    used_indices = used_indices.union(new_indices)
    
    class_counts = sub['repClass'].value_counts()
    if class_counts.size == 1:
        repclass = class_counts.index[0]
    else:
        raise
    
    norm_fam = ''.join([x.capitalize() for x in fam.split('_')])
    norm_class = ''.join([x.capitalize() for x in repclass.split('_')])
    
    print(norm_fam)
    print(norm_class)
        
    out_file = 'rmsk_cls-{}_fam-{}.bed'.format(norm_class, norm_fam)
    sub[['#genoName', 'genoStart', 'genoEnd', 'repName']].to_csv(
        os.path.join(outpath, out_file),
        sep='\t',
        header=False,
        index=False
    )

    
    
for cls in dump_classes:
    select_class = df['repClass'] == cls
    select_indices = ~df.index.isin(used_indices)
    sub = df.loc[select_class & select_indices, :]
    
    new_indices = set(sub.index)
    used_indices = used_indices.union(new_indices)
    
    family_counts = sub['repFamily'].value_counts()
    if family_counts.size == 1:
        repfam = family_counts.index[0]
    else:
        repfam = 'various'
    
    norm_fam = ''.join([x.capitalize() for x in repfam.split('_')])
    norm_class = ''.join([x.capitalize() for x in cls.split('_')])
    
    out_file = 'rmsk_cls-{}_fam-{}.bed'.format(norm_class, norm_fam)
    sub[['#genoName', 'genoStart', 'genoEnd', 'repName']].to_csv(
        os.path.join(outpath, out_file),
        sep='\t',
        header=False,
        index=False
    )

    


(5293264, 11)
Sva
Retroposon
LowComplexity
LowComplexity
SimpleRepeat
SimpleRepeat
Centr
Satellite
Acro
Satellite
Telo
Satellite
Satellite
Satellite
Alu
Sine


AttributeError: 'DataFrame' object has no attribute 'read_csv'